In [1]:
import pandas as pd
import vl_convert as vlc
from gprofiler import GProfiler
gp = GProfiler(return_dataframe=True)
from enum import Enum
import math
import altair as alt
import numpy as np

In [2]:
class canEnum(Enum):
    HNSCC = 0
    LSCC = 1
    CCRCC = 2
    LUAD = 3
    EN = 4

In [3]:
delta_correlation_df = pd.read_csv('data/delta_correlation_df_with_significance.csv')
delta_correlation_df

,Gene,Delta_Correlation,P_Value,FDR,Cancer,Significant
0,A1BG,-0.198013,2.451044e-01,4.045115e-01,HNSCC,False
1,A2M,-0.118384,4.480278e-01,6.091130e-01,HNSCC,False
2,A2ML1,-0.023469,2.918125e-01,4.561968e-01,HNSCC,False
3,AAAS,0.275905,1.051756e-01,2.209072e-01,HNSCC,False
4,AACS,-0.136836,1.800586e-01,3.266475e-01,HNSCC,False
...,...,...,...,...,...,...
50684,ZWINT,1.219024,2.267627e-09,1.049863e-07,Endometrial,True
50685,ZXDC,-0.346532,2.983295e-01,5.386144e-01,Endometrial,False
50686,ZYG11B,0.768196,5.463938e-04,5.319699e-03,Endometrial,True
50687,ZYX,0.253630,2.456049e-01,4.795301e-01,Endometrial,False


In [11]:
kegg_profiles = []
for cancer in pd.unique(delta_correlation_df.Cancer):
    
    cancer_df = delta_correlation_df[delta_correlation_df.Cancer == cancer]
    background_genes  = list(pd.unique(cancer_df.Gene))
    cancer_df = cancer_df[cancer_df.Significant == True]
    queryS = list(pd.unique(cancer_df.Gene))
    cancer_profile = gp.profile(organism='hsapiens', query = queryS, no_iea=True, sources = ["KEGG"],
                       ordered=True, no_evidences=False, background= background_genes)
    kegg_profiles.append(cancer_profile)

In [12]:
endo = kegg_profiles[canEnum.EN.value][['description','p_value']]
luad = kegg_profiles[canEnum.LUAD.value][['description','p_value']]
lscc = kegg_profiles[canEnum.LSCC.value][['description','p_value']]
hnscc = kegg_profiles[canEnum.HNSCC.value][['description','p_value']]
ccrcc = kegg_profiles[canEnum.CCRCC.value][['description','p_value']]

In [13]:
luad['Cancer'] = ['LUAD' for i in range(len(luad.index))]
endo['Cancer'] = ['Endometrial' for i in range(len(endo.index))]
lscc['Cancer'] = ['LSCC' for i in range(len(lscc.index))]
hnscc['Cancer'] = ['HNSCC' for i in range(len(hnscc.index))]
ccrcc['Cancer'] = ['CCRCC' for i in range(len(ccrcc.index))]

/var/folders/16/01vxw5fs66d9b01nlwf3w5yc0000gn/T/ipykernel_95035/956850580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  luad['Cancer'] = ['LUAD' for i in range(len(luad.index))]
/var/folders/16/01vxw5fs66d9b01nlwf3w5yc0000gn/T/ipykernel_95035/956850580.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  endo['Cancer'] = ['Endometrial' for i in range(len(endo.index))]
/var/folders/16/01vxw5fs66d9b01nlwf3w5yc0000gn/T/ipykernel_95035/956850580.py:3: SettingWithCopyWarning: 
A value is trying to be set o

In [14]:
kegg_df = pd.concat([endo,lscc,luad,ccrcc,hnscc])
kegg_df.p_value=-np.log(kegg_df.p_value)
kegg_df = kegg_df[kegg_df.description.isin(['Valine, leucine and isoleucine degradation', 
                                  'Fatty acid metabolism', 'p53 signaling pathway', 
                                  'Propanoate metabolism', 'beta-Alanine metabolism',
                                  'Glycolysis / Gluconeogenesis',
                                  'Butanoate metabolism', 'Glycerolipid metabolism'])]

In [15]:
chart = alt.Chart(kegg_df).mark_bar().encode(
    x = alt.X(
        'p_value',
        title = 'P value(Log)'
    ),
    y = alt.Y(
        'Cancer',
        title=None,
        axis=alt.Axis(
                labels=False,
                ticks=False,
                
            )
    ),
    color = 'Cancer',
    row=alt.Row(
        'description:O',
        sort = kegg_df['description'].values,
        spacing=0,
        header=alt.Header(
                title=None,
                labelAngle=0,
                labelAnchor="middle",
                labelOrient="left",
                labelAlign="left",labelFontSize=16
            )
    )
)
chart.configure_axis(
    titleFontSize=13
)
chart.configure_legend(
    titleFontSize=15,
    labelFontSize=13)

/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [16]:
png_data = vlc.vegalite_to_png(chart.to_json(), vl_version="4.17", scale=6)
with open("Figures/Figure_5_Cancer_Hallmarks_Pathways.png", "wb") as f:
    f.write(png_data)